# World cup 2022 forcast
Inspired by https://www.kaggle.com/code/agostontorok/soccer-world-cup-2018-winner/notebook  


Dataset:  
https://www.kaggle.com/datasets/piterfm/fifa-football-world-cup  
https://www.kaggle.com/datasets/cashncarry/fifaworldranking  
https://www.kaggle.com/datasets/brenda89/fifa-world-cup-2022  
https://www.kaggle.com/datasets/amineteffal/qatar2022worldcupschudule  
